In [1]:
import redis
import os
import environ
from pathlib import Path
import pickle
import pandas as pd

In [2]:
BASE_DIR = Path('__file__').resolve().parent
env = environ.Env()
env.read_env(os.path.join(BASE_DIR, ".env"))
HOST_NAME = env("HOST_NAME")
ACCESS_KEY = env("ACCESS_KEY")

In [3]:
r = redis.StrictRedis(host=HOST_NAME,
    port=6380, db=0, password=ACCESS_KEY, ssl=True)

In [4]:
r.ping()

True

In [5]:
df = pd.DataFrame(
    [
        ["北海道札幌市中央区北1条西2丁目", 50, 90],
        ["広島県廿日市市宮島町１−１", 100, 70],
        ["岐阜県不破郡関ケ原町関ケ原８９４−５５", 70, 50],
        ["大阪府大阪市中央区大阪城１−１", 30, 40],    
    ],
    columns=["住所", "座標x", "座標y"]
)

df

,住所,座標x,座標y
0,北海道札幌市中央区北1条西2丁目,50,90
1,広島県廿日市市宮島町１−１,100,70
2,岐阜県不破郡関ケ原町関ケ原８９４−５５,70,50
3,大阪府大阪市中央区大阪城１−１,30,40


In [6]:
speed = 50

for i in range(len(df)):
    for j in range(len(df)):
        X = df.iloc[i:i+1]
        Y = df.iloc[j:j+1]
#         display(X)
#         display(Y)

        # 起点
        start = X["住所"].values[0]
        print(start)

        # 終点
        end = Y["住所"].values[0]
        print(end)
        
        # 所要時間
        time = abs(X["座標x"].values[0]-Y["座標x"].values[0])*abs(X["座標y"].values[0]-Y["座標y"].values[0])/speed
        print(time)
        
        r.hset(start, end, time)

北海道札幌市中央区北1条西2丁目
北海道札幌市中央区北1条西2丁目
0.0
北海道札幌市中央区北1条西2丁目
広島県廿日市市宮島町１−１
20.0
北海道札幌市中央区北1条西2丁目
岐阜県不破郡関ケ原町関ケ原８９４−５５
16.0
北海道札幌市中央区北1条西2丁目
大阪府大阪市中央区大阪城１−１
20.0
広島県廿日市市宮島町１−１
北海道札幌市中央区北1条西2丁目
20.0
広島県廿日市市宮島町１−１
広島県廿日市市宮島町１−１
0.0
広島県廿日市市宮島町１−１
岐阜県不破郡関ケ原町関ケ原８９４−５５
12.0
広島県廿日市市宮島町１−１
大阪府大阪市中央区大阪城１−１
42.0
岐阜県不破郡関ケ原町関ケ原８９４−５５
北海道札幌市中央区北1条西2丁目
16.0
岐阜県不破郡関ケ原町関ケ原８９４−５５
広島県廿日市市宮島町１−１
12.0
岐阜県不破郡関ケ原町関ケ原８９４−５５
岐阜県不破郡関ケ原町関ケ原８９４−５５
0.0
岐阜県不破郡関ケ原町関ケ原８９４−５５
大阪府大阪市中央区大阪城１−１
8.0
大阪府大阪市中央区大阪城１−１
北海道札幌市中央区北1条西2丁目
20.0
大阪府大阪市中央区大阪城１−１
広島県廿日市市宮島町１−１
42.0
大阪府大阪市中央区大阪城１−１
岐阜県不破郡関ケ原町関ケ原８９４−５５
8.0
大阪府大阪市中央区大阪城１−１
大阪府大阪市中央区大阪城１−１
0.0


In [8]:
for i in range(len(df)):
    print( {k.decode("utf-8"):v.decode("utf-8") for k, v in r.hgetall(df["住所"].values[0]).items()})

{'北海道札幌市中央区北1条西2丁目': '0.0', '広島県廿日市市宮島町１−１': '20.0', '岐阜県不破郡関ケ原町関ケ原８９４−５５': '16.0', '大阪府大阪市中央区大阪城１−１': '20.0'}
{'北海道札幌市中央区北1条西2丁目': '0.0', '広島県廿日市市宮島町１−１': '20.0', '岐阜県不破郡関ケ原町関ケ原８９４−５５': '16.0', '大阪府大阪市中央区大阪城１−１': '20.0'}
{'北海道札幌市中央区北1条西2丁目': '0.0', '広島県廿日市市宮島町１−１': '20.0', '岐阜県不破郡関ケ原町関ケ原８９４−５５': '16.0', '大阪府大阪市中央区大阪城１−１': '20.0'}
{'北海道札幌市中央区北1条西2丁目': '0.0', '広島県廿日市市宮島町１−１': '20.0', '岐阜県不破郡関ケ原町関ケ原８９４−５５': '16.0', '大阪府大阪市中央区大阪城１−１': '20.0'}


In [10]:
speed = 50

for i in range(len(df)):
    for j in range(len(df)):
        X = df.iloc[i:i+1]
        Y = df.iloc[j:j+1]

        # 起点
        start = X["住所"].values[0]

        # 終点
        end = Y["住所"].values[0]
                
        print(start, end, r.hget(start, end).decode("utf-8"))

北海道札幌市中央区北1条西2丁目 北海道札幌市中央区北1条西2丁目 0.0
北海道札幌市中央区北1条西2丁目 広島県廿日市市宮島町１−１ 20.0
北海道札幌市中央区北1条西2丁目 岐阜県不破郡関ケ原町関ケ原８９４−５５ 16.0
北海道札幌市中央区北1条西2丁目 大阪府大阪市中央区大阪城１−１ 20.0
広島県廿日市市宮島町１−１ 北海道札幌市中央区北1条西2丁目 20.0
広島県廿日市市宮島町１−１ 広島県廿日市市宮島町１−１ 0.0
広島県廿日市市宮島町１−１ 岐阜県不破郡関ケ原町関ケ原８９４−５５ 12.0
広島県廿日市市宮島町１−１ 大阪府大阪市中央区大阪城１−１ 42.0
岐阜県不破郡関ケ原町関ケ原８９４−５５ 北海道札幌市中央区北1条西2丁目 16.0
岐阜県不破郡関ケ原町関ケ原８９４−５５ 広島県廿日市市宮島町１−１ 12.0
岐阜県不破郡関ケ原町関ケ原８９４−５５ 岐阜県不破郡関ケ原町関ケ原８９４−５５ 0.0
岐阜県不破郡関ケ原町関ケ原８９４−５５ 大阪府大阪市中央区大阪城１−１ 8.0
大阪府大阪市中央区大阪城１−１ 北海道札幌市中央区北1条西2丁目 20.0
大阪府大阪市中央区大阪城１−１ 広島県廿日市市宮島町１−１ 42.0
大阪府大阪市中央区大阪城１−１ 岐阜県不破郡関ケ原町関ケ原８９４−５５ 8.0
大阪府大阪市中央区大阪城１−１ 大阪府大阪市中央区大阪城１−１ 0.0
